# Que 8. For each hour of the day, what is the average number of seconds during which” torus receiver z+ input pipe errors” were detected and corrected?

basic steps followed to answer the questions were reffered from
https://spark.apache.org/docs/latest/api/python/getting_started/quickstart_df.html

In [1]:
#importing necessary libraries and datatypes

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DateType, TimestampType, IntegerType
from pyspark.sql.functions import *
import time

#setting up a spark session

session = SparkSession.builder.appName("BGLlog").getOrCreate()

23/08/05 23:02:09 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
# start time of the program
start_program_time= time.time()

# Schema Design

In [3]:
# defined the schema for the BGL log data

BGL_schema = StructType([

    StructField("AlertFlag", StringType(),True),
    
    # first Timestamp is assigned to IntegerType datatype and later to TimestampType datatype, 
    # as initializing Timestamp to "TimestampType" datatype in the beginning was not possible.
    StructField("Timestamp", IntegerType(),True), 

    StructField("Date", StringType(),True),

    StructField("Node", StringType(),True),

    StructField("date_and_time", TimestampType(),True),

    StructField("Node_Rep", StringType(),True),

    StructField("Message_Type", StringType(),True),

    StructField("SysComp", StringType(),True),

    StructField("Level", StringType(),True),

    StructField("Message_Content", StringType(),True)

])

In [4]:
# read the BGL log data from the CSV file with the specified schema (i.e. BGL_schema) 
# and date/timestamp formats are specified

BGL_df=session.read.csv(
    "BGLnew.log",
    schema = BGL_schema,
    dateFormat = "yyyy.MM.dd", 
    timestampFormat = "yyyy-MM-dd-HH.mm.ss.SSSSSS", 
    )

In [5]:
# cast the Timestamp coloumn to proper TimestampType 

BGL_df = BGL_df.withColumn("Timestamp", BGL_df["Timestamp"].cast(TimestampType()))

In [6]:
# print the schema to confirm the schema update of the Dataframe

BGL_df.printSchema()

root
 |-- AlertFlag: string (nullable = true)
 |-- Timestamp: timestamp (nullable = true)
 |-- Date: string (nullable = true)
 |-- Node: string (nullable = true)
 |-- date_and_time: timestamp (nullable = true)
 |-- Node_Rep: string (nullable = true)
 |-- Message_Type: string (nullable = true)
 |-- SysComp: string (nullable = true)
 |-- Level: string (nullable = true)
 |-- Message_Content: string (nullable = true)



# Transformation on BGL dataframe

In [7]:
# define the error message to be filtered

error_message = "torus receiver z+ input pipe error(s) (dcr 0x02f0) detected and corrected"

In [8]:
# filter the BGL Dataframe to get rows containing the specified error message

filtered_df = BGL_df.filter(col("Message_Content").contains(error_message))

In [9]:
# extract the duration in seconds from the "Message_Content" column using a regular expression

seconds_regex = r"over (\d+) seconds"
filtered_df = filtered_df.withColumn("seconds_detected", regexp_extract("Message_Content", seconds_regex, 1).cast("int"))

In [10]:
# create a new column "duration_seconds" to hold the extracted duration in seconds

filtered_df = filtered_df.withColumn("duration_seconds", col("seconds_detected"))

In [11]:
# extract the "hour_of_day" from the "date_and_time" column and store it in a new column

filtered_df = filtered_df.withColumn("hour_of_day", hour(from_utc_timestamp("date_and_time", "UTC")))

# Action on the filtered dataframe

In [12]:
# group the filtered Dataframe by "hour_of_day" and calculate the average of "duration_seconds" for each group

result_df = filtered_df.groupBy("hour_of_day").avg("duration_seconds")

In [13]:
# show the resulting Dataframe

result_df.show()

+-----------+---------------------+
|hour_of_day|avg(duration_seconds)|
+-----------+---------------------+
|         22|    9795.538461538461|
|          1|   13097.777777777777|
|          6|             362.0375|
|         20|              8286.25|
|         19|             4791.625|
|          8|            11041.875|
|         23|             6198.375|
|          7|    44635.21428571428|
|         10|            20594.375|
|         21|               4028.0|
|         11|   13411.857142857143|
|          0|   37279.666666666664|
|         12|   12036.785714285714|
|         13|    5202.933333333333|
|         16|   28040.846153846152|
|          3|              16011.9|
|         15|   16769.222222222223|
|         17|    8068.142857142857|
|          9|   3211.3333333333335|
|         14|   14729.368421052632|
+-----------+---------------------+
only showing top 20 rows



# Execution Time of the Program

In [14]:
# end time of the program
end_program_time= time.time()

# calculate executation time
end_program_time-start_program_time

6.864582777023315